In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
movies = pd.read_csv('movies.csv', usecols = ['movieId', 'title'])
rating = pd.read_csv('ratings.csv', usecols = ['userId', 'movieId', 'rating'])

In [3]:
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [4]:
rating.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [5]:
df = pd.merge(movies, rating, left_on = 'movieId', right_on = 'movieId', how = 'left')
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1.0,4.0
1,1,Toy Story (1995),5.0,4.0
2,1,Toy Story (1995),7.0,4.5
3,1,Toy Story (1995),15.0,2.5
4,1,Toy Story (1995),17.0,4.5


In [6]:
df.dropna(inplace = True)

In [7]:
moviemat = df.pivot_table(index = 'title', columns = 'userId', values = 'rating').fillna(0)
moviemat.head()

userId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,601.0,602.0,603.0,604.0,605.0,606.0,607.0,608.0,609.0,610.0
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
rating_count = pd.DataFrame(df.groupby('title')['rating'].count().reset_index().rename(columns = {'rating': 'rating_count'}))
rating_count.head()

,title,rating_count
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [9]:
df = df.merge(rating_count, on = 'title')
df.head()

,movieId,title,userId,rating,rating_count
0,1,Toy Story (1995),1.0,4.0,215
1,1,Toy Story (1995),5.0,4.0,215
2,1,Toy Story (1995),7.0,4.5,215
3,1,Toy Story (1995),15.0,2.5,215
4,1,Toy Story (1995),17.0,4.5,215


In [27]:
df = df.query('rating_count >= 50')
df.head()

In [10]:
from sklearn.neighbors import NearestNeighbors

In [12]:
movie_knn = NearestNeighbors(algorithm = 'brute', metric = 'cosine')
movie_knn.fit(moviemat)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [15]:
toy_story = moviemat.loc['Toy Story (1995)']
toy_story

userId
1.0      4.0
2.0      0.0
3.0      0.0
4.0      0.0
5.0      4.0
        ... 
606.0    2.5
607.0    4.0
608.0    2.5
609.0    3.0
610.0    5.0
Name: Toy Story (1995), Length: 610, dtype: float64

In [20]:
distances, indices = movie_knn.kneighbors([toy_story], n_neighbors = 10)

In [21]:
distances = distances[0]
indices = indices[0]